In [1]:
%load_ext autoreload
%autoreload 2

## load the trained model

In [2]:
import sys

In [3]:
sys.path.append('..') # append the parent directory of Transparency to python path

In [4]:
import os
import argparse
from collections import namedtuple
from Transparency.common_code.common import get_latest_model
from Transparency.configurations import configurations
from Transparency.Trainers.DatasetBC import datasets
from Transparency.model import Binary_Classification as BC

In [5]:
MockArgs = namedtuple(
    "MockArgs",
    [
        "dataset",
        "encoder",
        "data_dir",
        "output_dir",
        "attention",
        "diversity",
#         "seed",
#         "job_type",
    ],
)

In [8]:
args = MockArgs(dataset='sst',
                encoder='vanilla_lstm',
                data_dir='.',
                output_dir='../test_seeds',
                attention='tanh',
                diversity=0,
#                 seed=0,
#                 job_type='both'
               )

In [9]:
dataset = datasets[args.dataset](args)
config = configurations[args.encoder](dataset)

In [10]:
latest_model_name = get_latest_model(
    os.path.join(config["training"]["basepath"], config["training"]["exp_dirname"])
)

In [11]:
latest_model_name

'../test_seeds/sst/lstm+tanh/Tue_Jan_12_18:18:11_2021'

In [20]:
model = BC.Model.init_from_config(latest_model_name, load_gen=False)
model.dirname = latest_model_name      

encoder params {'vocab_size': 13826, 'embed_size': 300, 'type': 'vanillalstm', 'hidden_size': 128, 'pre_embed': None}


INFO - 2021-01-15 12:32:42,222 - instantiating class <class 'Transparency.model.modules.Encoder.Encoder'> from params {'vocab_size': 13826, 'embed_size': 300, 'type': 'vanillalstm', 'hidden_size': 128, 'pre_embed': None} and extras set()
INFO - 2021-01-15 12:32:42,224 - type = vanillalstm
INFO - 2021-01-15 12:32:42,225 - instantiating class <class 'Transparency.model.modules.Encoder.EncoderorthoRNN'> from params {'vocab_size': 13826, 'embed_size': 300, 'hidden_size': 128, 'pre_embed': None} and extras set()
INFO - 2021-01-15 12:32:42,227 - vocab_size = 13826
INFO - 2021-01-15 12:32:42,227 - embed_size = 300
INFO - 2021-01-15 12:32:42,228 - hidden_size = 128
INFO - 2021-01-15 12:32:42,228 - pre_embed = None
INFO - 2021-01-15 12:32:42,367 - instantiating class <class 'Transparency.model.modules.Decoder.AttnDecoder'> from params {'attention': {'type': 'tanh'}, 'output_size': 1, 'hidden_size': 256} and extras set()
INFO - 2021-01-15 12:32:42,368 - hidden_size = 256
INFO - 2021-01-15 12:32:

config  {'model': {'encoder': {}, 'decoder': {}, 'generator': {'hidden_size': 256, 'sparsity_lambda': 0.2}}, 'training': {'bsize': 32, 'weight_decay': 1e-05, 'pos_weight': [0.9135802469135803], 'basepath': '../test_seeds', 'exp_dirname': 'sst/lstm+tanh'}}
Running on device: cpu


In [22]:
predictions, attentions, conicity_values = model.evaluate(dataset.test_data.X)

In [25]:
conicity_values

array([0.47334918, 0.8559502 , 0.9091199 , ..., 0.8185079 , 0.7020547 ,
       0.33219284], dtype=float32)

In [33]:

def predict_proba(data):
    # TODO should have text here as first step
    predictions, _, _ = model.evaluate(dataset, data)
    return predictions

## use lime for explanations

In [29]:
# !pip install lime

In [30]:
from lime.lime_text import LimeTextExplainer

In [31]:
class_names = ['0', '1']

In [32]:
explainer = LimeTextExplainer(class_names=class_names)

In [34]:
idx = 83
exp = explainer.explain_instance([dataset.test_data.X[idx]], predict_proba, num_features=6)

/home/yun/.pyenv/versions/3.6.5/envs/fact/lib/python3.6/site-packages/lime/lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]


TypeError: expected string or bytes-like object